In [1]:
import numpy as np
np.random.seed(1001)

import os
import sys
import shutil

import IPython
import IPython.display as ipd  # To play sound in the notebook

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
# from tqdm import tqdm_notebook, tqdm
import tqdm

import librosa 
import wave

import torch 
import torchaudio
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split

from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv1d, MaxPool1d, Module, AdaptiveMaxPool1d
from torch.nn import Softmax, LogSoftmax, BatchNorm1d, Dropout, Flatten
from torch.nn import CrossEntropyLoss
from torch.optim import Adam, SGD

from torch.utils.tensorboard import SummaryWriter


dataset_path = '/home/renault/Projects/Statet-of-the-Art/Dataset/freesound-audio-tagging/'

/home/renault/env/lib/python3.5/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/renault/env/lib/python3.5/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/renault/env/lib/python3.5/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/renault/env/lib/python3.5/site-packages/tensorboard/compat/tensorflow_

## 1. Exploratory Data Analysis

### Load data

In [2]:
# ## load dataset and save as dataframe
# train = pd.read_csv(os.path.join(dataset_path,"train.csv"))
# test = pd.read_csv(os.path.join(dataset_path,"test_post_competition.csv"))

In [3]:
# print("Number of training examples=", train.shape[0], "  Number of classes in training set=", len(train.label.unique()))
# print("Number of test examples=", test.shape[0], "  Number of classes in test set=", len(test.label.unique()))
# train.head()

### Distribution of Categories

In [4]:
# category_group = train.groupby(['label', 'manually_verified']).count()
# plot = category_group.unstack().reindex(category_group.unstack().sum(axis=1).sort_values().index)\
#           .plot(kind='bar', stacked=True, title="Number of Audio Samples per Category", figsize=(16,10))
# plot.set_xlabel("Category")
# plot.set_ylabel("Number of Samples")
# print('Minimum samples per category = ', min(train.label.value_counts()))
# print('Maximum samples per category = ', max(train.label.value_counts()))

### Test listening audio 

In [5]:
# fname = dataset_path + 'audio_train/' + '00044347.wav'   # Hi-hat
# ipd.Audio(fname)

In [6]:
# ## plot wave using librosa 
# wav, sr = librosa.load(fname)
# print("Sampling (frame) rate = ", sr)
# print("Total samples (frames) = ", len(wav))
# print("Duration = ", len(wav)/sr)

# ## plot audio frames
# plt.plot(wav, 'r--')
# plt.title("Audio frames")
# plt.show()

# ## zoom in on first 500 frames
# plt.figure(figsize=(16,4))
# plt.plot(wav[:500],'--')
# plt.plot(wav[:500],'.')
# plt.title("First 500 frames")
# plt.show()

### Audio Length Distribution

In [7]:
# train['nframes'] = train['fname'].apply(lambda f: wave.open(dataset_path+'audio_train/' + f).getnframes())
# test['nframes'] = test['fname'].apply(lambda f: wave.open(dataset_path+'audio_test/' + f).getnframes())

In [8]:
# train.head()

In [9]:
# _, ax = plt.subplots(figsize=(16, 4))
# sns.violinplot(ax=ax, x="label", y="nframes", data=train)
# plt.xticks(rotation=90)
# plt.title('Distribution of audio frames, per label', fontsize=16)
# plt.show()

In [10]:
# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,5))
# train.nframes.hist(bins=100, ax=axes[0])
# test.nframes.hist(bins=100, ax=axes[1])
# plt.suptitle('Frame Length Distribution in Train and Test', ha='center', fontsize='large')

# 2. Building from scratch a simple CNN-1D using Raw Wave

### Loading audio using Torch Audio

In [11]:
# fname = dataset_path + 'audio_train/' + '00044347.wav'   # Hi-hat
# wav, sr = torchaudio.load(fname)  ## wav is now a tensor, not an array
# print("Sampling (frame) rate = ", sr)
# print("Total samples (frames) = ", wav.size(1))
# print("Duration = ", wav.size(1)/sr)

# ## plot audio frames
# plt.plot(wav.t().numpy()[:500], 'r--')
# plt.title("Audio frames")
# plt.show()

# # ## zoom in on first 500 frames
# # plt.figure(figsize=(16,4))
# # plt.plot(wav[:500],'--')
# # plt.plot(wav[:500],'.')
# # plt.title("First 500 frames")
# # plt.show()

# resampled_wav = torchaudio.transforms.Resample(sr, 16000)(wav)
# print(resampled_wav.t().size())
# print(resampled_wav.size(1))
# plt.plot(resampled_wav.t()[:500].numpy())
# plt.show()

### Prepare training dataset

In [12]:
# classes = train['label'].unique()
# print(classes)
# list(classes).index('Saxophone')

In [13]:
class AudioDataset(Dataset):
    
    def __init__(self, csv_file, root_dir=None, sampling_rate=44100, is_labeled = True, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.sampling_rate = sampling_rate
        self.is_labeled = is_labeled
        
        print("Nb of training samples = ", self.__len__())
    
        if self.is_labeled :
            self.all_classes = list(self.data['label'].unique())
            self.n_classes = len(self.all_classes)
            print('Nb of classes = ', self.n_classes)

    ## len(dataset) return the size of dataset    
    def __len__(self):
        return self.data.shape[0]
    
    def get_classes(self):
        return self.all_classes
    
    ## dataset[i] return the ith sample
    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        fname = os.path.join(self.root_dir,
                            self.data.iloc[idx,0])
        
        wav, sr = torchaudio.load(fname)  ## wav is a tensor, not an array
        ## Resampling with new sampling_rate
        sample = torchaudio.transforms.Resample(sr, self.sampling_rate)(wav)
#         sample = wav.t()
        
        ## Random offset + Padding + Normalization
        if self.transform :
            sample = self.transform(sample)

        if self.is_labeled:
            label_name = self.data.iloc[idx,1]
            label = self.all_classes.index(label_name)
            return {'raw_signal':sample,'label':label}
            
        else : 
            return {'raw_signal':sample}

    
        
# ### Test 

# audio_ds= AudioDataset(csv_file = os.path.join(dataset_path,"train.csv"),
#                       root_dir  = dataset_path + 'audio_train/',
#                       sampling_rate = 16000,
#                       is_labeled = True)
# classes = audio_ds.get_classes()
# print(len(audio_ds))
# for i in range(3):
#     sample = audio_ds[i]
#     print(sample['raw_signal'].size())
#     ## plot audio frames
#     plt.plot(sample['raw_signal'].t().numpy(), 'r--')
#     plt.title("Audio frames " + str(i) + ": " + classes[(sample['label'])])
#     plt.show()

class Random_Sample(object):
    
    def __init__(self, sample_duration, sampling_rate):
        self.sample_length = sample_duration * sampling_rate

    def __call__(self, sample):
        original_audio_length = sample.size(1)
        sample = sample.t()
        ## random offset / Padding
        if original_audio_length > self.sample_length:
            max_offset = original_audio_length - self.sample_length
            offset = np.random.randint(max_offset)
            new_sample = sample[offset:(offset+self.sample_length)].numpy()
        else:
            if original_audio_length == self.sample_length:
                offset = 0
            else: 
                max_offset = self.sample_length - original_audio_length
                offset = np.random.randint(max_offset)
            ## zero padding 
            new_sample = np.pad(sample.numpy()[:,0], (offset, self.sample_length - original_audio_length - offset), "constant")
                
        ## Normalize
        max_val = np.max(new_sample)
        min_val = np.min(new_sample)
    
        new_sample = (new_sample - min_val)/(max_val - min_val + 1e-6)
        new_sample -= 0.5
        
        return torch.from_numpy(new_sample.reshape(1,-1))

### Test 

transformed_audio_ds= AudioDataset(csv_file = os.path.join(dataset_path,"train.csv"),
                      root_dir  = dataset_path + 'audio_train/',
                      sampling_rate = 16000,
                      transform=Random_Sample(sample_duration=2,sampling_rate=16000))
classes = transformed_audio_ds.get_classes()
    

Nb of training samples =  9473
Nb of classes =  41


In [14]:
# # print(len(transformed_audio_ds))
# for i in range(3):
#     sample = transformed_audio_ds[i]
#     print(sample['raw_signal'].size())
#     ## plot audio frames
#     plt.plot(sample['raw_signal'].t().numpy(), 'r--')
#     plt.title("Audio frames " + str(i) + ": " + classes[(sample['label'])])
#     plt.show()


### Iterating through dataset using DataLoader

In [15]:
# def show_raw_wave_batch(sample_batched):
#     batch_size = sample_batched['raw_signal'].size(0)
#     for i in range(batch_size):
#         plt.plot(sample_batched['raw_signal'][i].t().numpy())
#         plt.show()

In [16]:
# batch_size = 3
# dataloader = DataLoader(transformed_audio_ds, batch_size = batch_size, shuffle =True, num_workers=4)

# # for i_batch, sample_batched in enumerate(dataloader):
# #     data, label = sample_batched['raw_signal'], sample_batched['label']
# #     print(i_batch, data.size(), label)
# #     if i_batch == 3 :
# #         show_raw_wave_batch(sample_batched)
# #         break
# dataloader = iter(dataloader)
# len(dataloader)


In [17]:
# sampled_batch = next(dataloader)
# print(sampled_batch['label'])

### Build model

In [18]:
# class Dummy_Conv1D_Net(Module):
    
#     def __init__(self):
#         super(Dummy_Conv1D_Net,self).__init__()
        
#         self.cnn_layers = Sequential(
#             ## First conv bloc
#             Conv1d(in_channels=1, out_channels=16, kernel_size=9, padding=0),
#             ReLU(inplace=True),
#             Conv1d(in_channels=16, out_channels=16, kernel_size=9, padding=0),
#             ReLU(inplace=True),
#             MaxPool1d(kernel_size=16),
#             Dropout(p=0.1),
#             Flatten()
            
#         )
        
#         self.linear_layers = Sequential(
#             Linear(in_features = 16*1999, out_features = 1024),
#             Linear(in_features = 1024, out_features = 41)
#         )
    
#     def forward(self,x):
#         x = self.cnn_layers(x)
# #         x = x.view((x.size(0), -1)) ## Flatten
# #         print(x.size())
#         x = self.linear_layers(x)
#         return x

# dummy_model = Dummy_Conv1D_Net()
# print(dummy_model)

In [19]:
class Conv1D_Net(Module):
    
    def __init__(self):
        super(Conv1D_Net,self).__init__()
        
        self.cnn_bloc_1 = Sequential(
            ## First conv bloc
            Conv1d(in_channels=1, out_channels=16, kernel_size=9, padding=0),
            ReLU(),
            Conv1d(in_channels=16, out_channels=16, kernel_size=9, padding=0),
            ReLU(),
            MaxPool1d(kernel_size=16),
            Dropout(p=0.1)            
        )
        
        self.cnn_bloc_2 = Sequential(
            ## second conv bloc
            Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=0),
            ReLU(),
            Conv1d(in_channels=32, out_channels=32, kernel_size=3, padding=0),
            ReLU(),
            MaxPool1d(kernel_size=4),
            Dropout(p=0.1)            
        )
        
        self.cnn_bloc_3 = Sequential(
            ## Third conv bloc
            Conv1d(in_channels=32, out_channels=32, kernel_size=3, padding=0),
            ReLU(),
            Conv1d(in_channels=32, out_channels=32, kernel_size=3, padding=0),
            ReLU(),
            MaxPool1d(kernel_size=4),
            Dropout(p=0.1)
        )
        
        self.cnn_bloc_4 = Sequential(
            ## Fourth conv bloc
            Conv1d(in_channels=32, out_channels=256, kernel_size=3, padding=0),
            ReLU(),
            Conv1d(in_channels=256, out_channels=256, kernel_size=3, padding=0),
            ReLU(),
            AdaptiveMaxPool1d(output_size=1),
            Dropout(p=0.2),
            Flatten()
        )
        
        self.linear_layers = Sequential(
            Linear(in_features = 256, out_features = 64),
            ReLU(),
            Linear(in_features = 64, out_features = 1028),
            ReLU(),
            Linear(in_features = 1028, out_features = 41)
#             Softmax(dim=1)
        )
    
    def forward(self,x):
        x = self.cnn_bloc_1(x)
        x = self.cnn_bloc_2(x)
        x = self.cnn_bloc_3(x)
        x = self.cnn_bloc_4(x)
        x = self.linear_layers(x)
        return x

conv1d_model = Conv1D_Net()
print(conv1d_model)

## Define loss function an optimizer
criterion = CrossEntropyLoss()
optimizer = Adam(conv1d_model.parameters(), lr=0.001, weight_decay=1e-5)

Conv1D_Net(
  (cnn_bloc_1): Sequential(
    (0): Conv1d(1, 16, kernel_size=(9,), stride=(1,))
    (1): ReLU()
    (2): Conv1d(16, 16, kernel_size=(9,), stride=(1,))
    (3): ReLU()
    (4): MaxPool1d(kernel_size=16, stride=16, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout(p=0.1, inplace=False)
  )
  (cnn_bloc_2): Sequential(
    (0): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
    (1): ReLU()
    (2): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
    (3): ReLU()
    (4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout(p=0.1, inplace=False)
  )
  (cnn_bloc_3): Sequential(
    (0): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
    (1): ReLU()
    (2): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
    (3): ReLU()
    (4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout(p=0.1, inplace=False)
  )
  (cnn_bloc_4): Sequential(
    (0): Conv1d(32, 256, kernel_size=(3,), stride=(1,))
    (1): ReL

### TensorBoard setup

In [20]:
writer = SummaryWriter('runs/conv1d_audio_test_2')
# writer.close()

### Train model

In [21]:
train_ds, val_ds = random_split(transformed_audio_ds, [8053,1420])

batch_size = 64

train_dataloader = DataLoader(train_ds, batch_size = batch_size, shuffle =True, num_workers=10)
val_dataloader = DataLoader(val_ds, batch_size = batch_size, shuffle =True, num_workers=10)
train_batches = len(train_dataloader)
val_batches = len(val_dataloader)
# train_iter  = iter(train_dataloader)
# val_iter  = iter(val_dataloader)
# data, labels = next(iter(dataloader))
print(train_batches, val_batches)

126 23


In [22]:
# sample = next(train_iter)
# sample['label']

In [23]:
# for idx, sample in enumerate(train_dataloader):
#     print(idx, sample['label'])
#     if idx==3:
#         break

In [24]:
## Train model

# print("Adding model to tensorboard")
# sample_batched = next(train_dataloader)
# data, label = sample_batched['raw_signal'], sample_batched['label']
# writer.add_graph(conv1d_model, data)

model_added =  False 
num_epochs = 50
n_iter = 0
best_acc_loss = np.inf
checkpoint = train_batches

print("Start training...")
for epoch in range(num_epochs):
    train_losses = []
    train_accs = []
    val_losses = []
    val_accs  = []

#     train_iter  = iter(train_dataloader)
#     val_iter  = iter(val_dataloader)
    
    print('Epoch %d/%d :' %(epoch +1, num_epochs))
#     print('[Training phase] : ', end='')
    
    for i_batch, sample_batched in tqdm.notebook.tqdm(enumerate(train_dataloader), total=train_batches):
#         sample_batched = next(train_iter)
        n_iter += 1
        data, label = sample_batched['raw_signal'], sample_batched['label']
        
        if not model_added:
            print("Adding model to tensorboard...\n")
            writer.add_graph(conv1d_model, data)
            model_added = True
            
#         conv1d_model.train()
        
        ## forward + backward + optimize
        outputs = conv1d_model(data)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        ## compute statistics
        running_loss = loss.item()
        _, y_pred = torch.max(F.softmax(outputs, dim=1),1)
#         print(y_pred, "vs ",label)
        correct = (y_pred == label).sum().item()
        running_acc = correct/label.size(0)
        
        
        train_losses.append(running_loss)
        train_accs.append(running_acc)
        
        sys.stdout.write("\rcurrent_train_loss = %.5f, current_train_acc = %.5f" 
                         % (np.mean(train_losses), np.mean(train_accs)))
        sys.stdout.flush()

        #=======================
        # print every a number n iterations + validate model
        #=======================        
        if n_iter % checkpoint == 0:
            train_loss = np.mean(train_losses)
            train_acc = np.mean(train_accs)
        
            writer.add_scalar('train/loss', train_loss, n_iter)
            writer.add_scalar('train/acc', train_acc, n_iter)
                    
            ## valid model
#             print("[Validating phase] :", end ='')
            print('\nValidating...')
            conv1d_model.eval()
            with torch.no_grad():
                for id_val_batch, val_batched in enumerate(val_dataloader):
        #             val_batched = next(val_iter)
                    val_data, val_label = val_batched['raw_signal'], val_batched['label']
#                     conv1d_model.eval()
                    val_outputs = conv1d_model(val_data)
                    val_loss = criterion(val_outputs, val_label).item()
                    _, val_pred = torch.max(F.softmax(val_outputs, dim=1),1)
                    val_correct = (val_pred == val_label).sum().item()
                    val_acc = val_correct / val_label.size(0)

#                     sys.stdout.write("\r val_loss = %.5f,  val_acc = %.5f" % (val_loss, val_acc))
#                     sys.stdout.flush()

                    val_losses.append(val_loss)
                    val_accs.append(val_acc)
                
            val_loss = np.mean(val_losses)
            val_acc  = np.mean(val_accs)
#             print('val_loss = %.5f, val_acc = %.5f' % (val_loss, val_acc))
            writer.add_scalar('val/loss', val_loss, n_iter)
            writer.add_scalar('val/acc', val_acc, n_iter)

            print("val_loss = %.5f, val_acc = %.5f\n" 
                             % (val_loss, val_acc))
#             sys.stdout.flush()
            if val_loss < best_acc_loss : 
                best_acc_loss = val_loss
                sys.stdout.write("Saving model...\n")
                torch.save(conv1d_model, 'conv1d_test.pt')
                
            conv1d_model.train()
            train_losses = []
            train_accs = []
            val_losses = []
            val_accs  = []
            
#     writer.add_scalar('train/loss', train_loss, epoch)
#     writer.add_scalar('train/acc', train_acc, epoch)
#     writer.add_scalar('val/loss', val_loss, epoch)
#     writer.add_scalar('val/acc', val_acc, epoch)
    
    print("=" * 80 )
    
print("Finish training.")
writer.close()

Start training...
Epoch 1/50 :


Adding model to tensorboard...

current_train_loss = 3.66625, current_train_acc = 0.02865
Validating...
train_loss = 3.66625,  train_acc = 0.02865, val_loss = 3.66206, val_acc = 0.03125

Saving model...

Epoch 2/50 :


/home/renault/env/lib/python3.5/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Conv1D_Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


current_train_loss = 3.65437, current_train_acc = 0.03182
Validating...
train_loss = 3.65437,  train_acc = 0.03182, val_loss = 3.58843, val_acc = 0.05729

Saving model...

Epoch 3/50 :


current_train_loss = 3.34249, current_train_acc = 0.07361
Validating...
train_loss = 3.34249,  train_acc = 0.07361, val_loss = 3.19667, val_acc = 0.09692

Saving model...

Epoch 4/50 :


...

current_train_loss = 1.76307, current_train_acc = 0.47730
Validating...
train_loss = 1.76307,  train_acc = 0.47730, val_loss = 1.84891, val_acc = 0.45312


Epoch 47/50 :


current_train_loss = 1.74546, current_train_acc = 0.48318
Validating...
train_loss = 1.74546,  train_acc = 0.48318, val_loss = 1.80447, val_acc = 0.48958

Saving model...

Epoch 48/50 :


current_train_loss = 1.73239, current_train_acc = 0.48762
Validating...
train_loss = 1.73239,  train_acc = 0.48762, val_loss = 1.83948, val_acc = 0.47532


Epoch 49/50 :


current_train_loss = 1.74222, current_train_acc = 0.48546
Validating...
train_loss = 1.74222,  train_acc = 0.48546, val_loss = 1.83707, val_acc = 0.47101


Epoch 50/50 :


current_train_loss = 1.71756, current_train_acc = 0.49031
Validating...
train_loss = 1.71756,  train_acc = 0.49031, val_loss = 1.81660, val_acc = 0.46966


Finish training.


In [25]:
# x =  0
# for i_batch, sample in tqdm.notebook.tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
#     label = sample['label']
#     print(label)

In [26]:
torch.cuda.is_available()

False

### Load pretrained model 

In [33]:
load_conv1d_model = torch.load('conv1d_test.pt')
load_conv1d_model.eval()
for id_val_batch, val_batched in enumerate(val_dataloader):
        
    val_data, val_label = val_batched['raw_signal'], val_batched['label']
    val_outputs = load_conv1d_model(val_data)
    val_loss = criterion(val_outputs, val_label).item()
    _, val_pred = torch.max(F.softmax(val_outputs, dim=1),1)
    val_correct = (val_pred == val_label).sum().item()
    val_acc = val_correct / val_label.size(0)

    print(val_loss)
    print(val_acc)
    
    break

1.8254387378692627
0.453125
